In [44]:
%load_ext autoreload
%autoreload 2

import os

from open_hardware_definitions import *
from open_hardware_definitions.common import DATASHEET_DIR, DEFINITIONS_DIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
# You can get these files from the "HighTec Free TriCore™ Entry Tool Chain"
# https://free-entry-toolchain.hightec-rt.com/

NAME = "tc29xb"
HEADER_FOLDER = f"{DATASHEET_DIR}/{NAME}"

header_files = os.listdir(HEADER_FOLDER)
reg_files = list(filter(lambda x: '_reg.h' in x and x != "Ifx_reg.h", header_files))
regdef_files = list(filter(lambda x: '_regdef.h' in x, header_files))
bf_files = list(filter(lambda x: '_bf.h' in x, header_files))

modules = list(map(lambda x: x.split('_')[0].split("Ifx")[1], reg_files))

In [51]:
dev = Device(
  manufacturer="Infineon",
  part_number=NAME,
  architecture="TriCore",
  bit_width=32,
  endianness=Endianness.LITTLE,
)

dev.regions = [
  Region("SRAM", 0x50000000, 0x80000000 - 0x50000000, readable=True, writable=True, executable=True, volatile=False),
  Region("PFLASH1", 0x80000000, 0x80800000 - 0x80000000, readable=True, writable=False, executable=True, volatile=False),
  Region("PFLASH2", 0xA0000000, 0xA0800000 - 0x80000000, readable=True, writable=False, executable=True, volatile=False),
  Region("BOOTROM1", 0x8FFF8000, 0x90000000 - 0x8FFF8000, readable=True, writable=False, executable=True, volatile=False),
  Region("BOOTROM2", 0xAFFF8000, 0xB0000000 - 0xAFFF8000, readable=True, writable=False, executable=True, volatile=False),
  Region("BOOTROM_SCRATCHPAD", 0xC0000000, 0xD0000000 - 0xC0000000, readable=True, writable=True, executable=True, volatile=False),
]

In [52]:
dev.modules = []
for fn in reg_files:
  m = Module(
    name=fn.split('_')[0].split("Ifx")[1].upper(),
    registers=[],
  )

  with open(f"{HEADER_FOLDER}/{fn}", "r") as f:
    for l in f:
      if "#define" in l and "0x" in l:
        reg_name = l.split(' ')[1]
        try:
          reg_addr = int(l.split('u)')[-2].split('0x')[1], 16)
        except:
          reg_addr = int(l.split(' ')[-1], 16)
        m.registers.append(Register(
          name=reg_name,
          addr=reg_addr,
          size_bits=32, # TODO: is this generally correct?
        ))

  m.base_addr = min([r.addr for r in m.registers])
  m.size = max([r.addr for r in m.registers]) + 4 - m.base_addr

  dev.modules.append(m)


In [ ]:
# TODO: parse structs and bitfields

In [43]:
print(dev.dump())

!Device
manufacturer: Infineon
part_number: TC29x
architecture: TriCore
bit_width: 32
endianness: little
regions:
- !Region
  name: SRAM
  base_addr: 0x50000000
  size: 0x30000000
  readable: true
  writable: true
  executable: true
  volatile: false
- !Region
  name: PFLASH1
  base_addr: 0x80000000
  size: 0x800000
  readable: true
  writable: false
  executable: true
  volatile: false
- !Region
  name: PFLASH2
  base_addr: 0xa0000000
  size: 0x20800000
  readable: true
  writable: false
  executable: true
  volatile: false
- !Region
  name: BOOTROM1
  base_addr: 0x8fff8000
  size: 0x8000
  readable: true
  writable: false
  executable: true
  volatile: false
- !Region
  name: BOOTROM2
  base_addr: 0xafff8000
  size: 0x8000
  readable: true
  writable: false
  executable: true
  volatile: false
- !Region
  name: BOOTROM_SCRATCHPAD
  base_addr: 0xc0000000
  size: 0x10000000
  readable: true
  writable: true
  executable: true
  volatile: false
modules:
- !Module
  name: DSADC
  base_ad

In [53]:
# Commit to file
with open(DEFINITIONS_DIR + f"/{dev.manufacturer}/{dev.part_number}.yaml", 'w') as f:
  f.write(dev.dump())